In [1]:
from GameEnvironment import ModelGameEnvironment
import torch
from ActorCritic import ActorCritic, Actor, Critic

In [2]:
game = ModelGameEnvironment(.5, 2, .99, 10)

In [3]:
game.reset()


(tensor([50., 50.]), None, False)

In [4]:
print(game.step(torch.Tensor([10,10]) ))
print(f"quantity history {game.quantity}")
print(f"profit history {game.profit}")

(tensor([10., 10.]), tensor([-195., -195.]), False)
quantity history tensor([[10.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [10.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]])
profit history tensor([[-195.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.],
        [-195.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.]])


In [5]:
env = ModelGameEnvironment(.5, 2, .99, 10)

# Initialize actor and critic
input_dim = 2

actor = Actor(input_dim)
critic = Critic(input_dim)

# Initialize actor-critic algorithm
agent = ActorCritic(actor, critic, .99)

# Training loop
num_episodes = 1000
for episode in range(num_episodes):
    state, _, done = env.reset()

    
    while not done:
        action_profile = torch.Tensor(2)
        log_probs = []


        for _ in range(2):
            action, log_prob = agent.select_action(state)
            action_profile[_] = action
            log_probs.append(log_prob)
        
        
        # Compute reward based on the action profile
        next_state, reward_tensor, done = env.step(action_profile)
        reward = torch.sum(reward_tensor).item()
        


        #for i in range(2):
        agent.update(state,  reward, log_probs[0], next_state)
        


    if episode % 100 == 0:
        print(f"Episode {episode} complete")
        print(torch.sum(env.profit))

Episode 0 complete
tensor(-1.1477e+11)
Episode 100 complete
tensor(-27.0803)
Episode 200 complete
tensor(-25.5474)
Episode 300 complete
tensor(-25.3987)
Episode 400 complete
tensor(-25.5567)
Episode 500 complete
tensor(-23.7390)
Episode 600 complete
tensor(-11.0857)
Episode 700 complete
tensor(-10.6153)
Episode 800 complete
tensor(-9.8363)
Episode 900 complete
tensor(-8.8265)
